# Ursnif Malware Download URL Pattern
Detects download of Ursnif malware done by dropper documents.

## Rule Content
```
- title: Ursnif Malware Download URL Pattern
  id: a36ce77e-30db-4ea0-8795-644d7af5dfb4
  status: stable
  description: Detects download of Ursnif malware done by dropper documents.
  author: Thomas Patzke
  logsource:
    category: proxy
    product: null
    service: null
  detection:
    selection:
      c-uri: '*/*.php?l=*.cab'
      sc-status: 200
    condition: selection
  fields:
  - c-ip
  - c-uri
  - sc-bytes
  - c-ua
  falsepositives:
  - Unknown
  level: critical
- title: Ursnif Malware C2 URL Pattern
  id: 932ac737-33ca-4afd-9869-0d48b391fcc9
  status: stable
  description: Detects Ursnif C2 traffic.
  references:
  - https://www.fortinet.com/blog/threat-research/ursnif-variant-spreading-word-document.html
  author: Thomas Patzke
  logsource:
    category: proxy
  detection:
    b64encoding:
      c-uri:
      - '*_2f*'
      - '*_2b*'
    urlpatterns:
      c-uri|all:
      - '*.avi'
      - '*/images/*'
    condition: b64encoding and urlpatterns
  fields:
  - c-ip
  - c-uri
  - sc-bytes
  - c-ua
  falsepositives:
  - Unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(c-uri.keyword:*\/*.php?l\=*.cab AND sc-status:"200")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(c-uri.keyword:(*_2f* OR *_2b*) AND c-uri.keyword:*.avi AND c-uri.keyword:*\/images\/*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()